In [11]:
from datetime import timedelta, date
from supabase import create_client, Client
from dotenv import load_dotenv
import os


load_dotenv()

url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")
supabase = create_client(url, key)

usage = supabase.table("disposable_daily_usage").select("*").execute().data
print(usage)

[{'disposable_id': 4, 'disposable_name': 'Hot Cup Sleeves', 'log_date': '2025-07-14', 'quantity_used': 90}, {'disposable_id': 10, 'disposable_name': 'Syrup Cup 2oz', 'log_date': '2025-07-14', 'quantity_used': 12}, {'disposable_id': 9, 'disposable_name': 'Round Bowl Lid 16oz', 'log_date': '2025-07-14', 'quantity_used': 47}, {'disposable_id': 7, 'disposable_name': 'Sandwich Box', 'log_date': '2025-07-14', 'quantity_used': 40}, {'disposable_id': 15, 'disposable_name': 'Soup Bowl 10oz', 'log_date': '2025-07-14', 'quantity_used': 16}, {'disposable_id': 6, 'disposable_name': 'Cold Cup 16oz', 'log_date': '2025-07-14', 'quantity_used': 173}, {'disposable_id': 12, 'disposable_name': 'Espresso Cup 4oz', 'log_date': '2025-07-14', 'quantity_used': 5}, {'disposable_id': 24, 'disposable_name': 'Hot Dog Sleeve', 'log_date': '2025-07-14', 'quantity_used': 309}, {'disposable_id': 19, 'disposable_name': 'Plate 9"', 'log_date': '2025-07-14', 'quantity_used': 199}, {'disposable_id': 14, 'disposable_name':

In [ ]:
yesterday = date.today() - timedelta(days=1)

for entry in usage:
    disp_id = entry["disposable_id"]
    used_qty = entry["quantity_used"]
    disp_name = entry["disposable_name"]

    supabase.table("disposable_usage_log").insert({
    "disposable_name":disp_name,
    "disposable_id": disp_id,
    "quantity_used": used_qty,
    "log_date": str(yesterday)
        }).execute()
    
    
    # Executing SQL Stored procedure to deduct stock
    
    supabase.rpc(
        "update_stock_after_sales",
        {   
            "disp_id":disp_id,
            "used_qty": used_qty
         }).execute()
    

In [3]:
from datetime import datetime, timedelta
FILE_DATE = (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d")
print(FILE_DATE)

2025-07-23


In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime, timedelta
from supabase import create_client, Client
from io import BytesIO


load_dotenv()  # loading variables from .env


# ========== CONFIG ==========
# DATA_FOLDER = "data"
# CLEAN_DATA_FOLDER = "clean_data"
# FILE_DATE = (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d")
# INPUT_FILE = f"../{DATA_FOLDER}/{FILE_DATE}.xlsx"
# CLEANED_FILE = f"../{CLEAN_DATA_FOLDER}/{FILE_DATE}_cleaned.csv"

SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

download_bucket = os.environ.get("FOOD_SALES_BUCKET")
upload_bucket = os.environ.get("CLEAN_BUCKET")

FILE_DATE = (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d")

INPUT_FILE_NAME = f"{FILE_DATE}.xlsx"
OUTPUT_FILE_NAME = f"{FILE_DATE}_cleaned.csv"

raw_file = supabase.storage.from_(download_bucket).download(INPUT_FILE_NAME)

print(raw_file)

StorageApiError: {'statusCode': 404, 'error': Bucket not found, 'message': Bucket not found}